In [ ]:
import os

def scale_input_data(scale_factor):
  file_bases = ['./input/training_set', './input/training_set_metadata', './input/test_set_metadata']
  for file_base in file_bases:
    import pandas as pd
    import shutil
    if scale_factor == 1.0:
      shutil.copyfile(file_base + '.csv', file_base + '.scaled.csv')
      continue
    df_to_scale = pd.read_csv(file_base + '.csv')
    new_num_rows = int(scale_factor * len(df_to_scale))
    if scale_factor <= 1.0:
      df_to_scale = df_to_scale.iloc[:new_num_rows]
    else:
      while len(df_to_scale) < new_num_rows:
        df_to_scale = pd.concat([df_to_scale, df_to_scale[:min(new_num_rows - len(df_to_scale), len(df_to_scale))]])
    df_to_scale.to_csv(file_base + '.scaled.csv', index=False)

if 'INPUT_SCALE_FACTOR' in os.environ:
  scale_input_data(float(os.environ['INPUT_SCALE_FACTOR']))

In [1]:
# FIRST-AUTHOR: remove extra display code, path printing
# from datetime import datetime
# import os

import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
exec(os.environ['IREWR_IMPORTS'])
# FIRST-AUTHOR: remove ML code
# from scipy.stats import skew

from IPython.core.display import display
from tqdm import tqdm
tqdm.pandas()

# FIRST-AUTHOR: remove path printing
# print(os.listdir("./input"))

/tmp/ipykernel_81175/2634013812.py:10: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display


pivot_tableを用いた高速データ処理
==========================

# TL;DR
カテゴリごとの集計をしたいときにpivot_tableを用いると直感的で高速な処理ができる

In [2]:
# 冒頭のpivot_tableを用いたバージョン
# 欠損値が含まれることに注意する

# FIRST-AUTHOR: remove extra display code
# tick = datetime.now()
train_df = pd.read_csv("./input/training_set.scaled.csv", dtype={"object_id": np.uint32,
                                                           "mjd": np.float64,
                                                           "passband": np.uint8,
                                                           "flux": np.float32,
                                                           "flux_err": np.float32,
                                                           "detected": np.uint8})
train_meta_df = pd.read_csv('./input/training_set_metadata.scaled.csv')
# FIRST-AUTHOR: remove extra display code
# tock = datetime.now()
# print("load_data: {} ms".format((tock - tick).seconds * 1000 + ((tock - tick).microseconds / 1000)))

# tick = datetime.now()

# pivot_tableのindexをrankを用いて作成する
train_df["rank"] = train_df.groupby(["object_id", "passband"])["mjd"].rank()

flux = train_df.pivot_table(columns=["object_id", "passband"],
                            index="rank",
                            values="flux",
                            aggfunc="mean")
dflux = train_df.pivot_table(columns=["object_id", "passband"],
                             index="rank",
                             values="flux_err",
                             aggfunc="mean")

# 列にNaNが含まれるので扱いに注意する
flux_mean = np.sum(flux*np.square(flux/dflux), axis=0)/np.sum(np.square(flux/dflux), axis=0)
flux_std = np.std(flux/flux_mean, ddof = 1, axis=0)
flux_amp = (np.max(flux, axis=0) - np.min(flux, axis=0))/flux_mean
flux_mad = np.nanmedian(np.abs((flux - np.nanmedian(flux, axis=0))/flux_mean), axis=0) # array
flux_beyond = np.sum(np.abs(flux - flux_mean) > np.std(flux, ddof = 1, axis=0), axis=0)/flux.count()
# FIRST-AUTHOR: remove ML code
# flux_skew = skew(flux, nan_policy="omit", axis=0)  # masked_array
flux_skew = 0.0

result_df = pd.concat([flux_mean.reset_index(name="flux_mean"),
                      flux_std.reset_index(name="flux_std").iloc[:, 2:],
                      flux_amp.reset_index(name="flux_amp").iloc[:, 2:],
                      flux_beyond.reset_index(name="flux_beyond").iloc[:, 2:]], axis=1)
result_df["flux_mad"] = flux_mad
result_df["flux_skew"] = flux_skew
colnames = ["flux_mean", "flux_std", "flux_amp", "flux_beyond", "flux_mad", "flux_skew"]

for j in range(6):
    train_meta_df = train_meta_df.merge(result_df.loc[result_df.passband == j, :]
                                                 .rename(columns={colname: "{}_{}".format(colname, j) for colname in colnames})
                                                 .drop("passband", axis=1),
                                        how="left",
                                        on=["object_id"])
# FIRST-AUTHOR: remove extra display code
# tock = datetime.now()
# print("processing_time: {} sec".format((tock - tick).seconds))

train_meta_df.head()

,object_id,ra,decl,gal_l,gal_b,ddf,hostgal_specz,hostgal_photoz,hostgal_photoz_err,distmod,...,flux_amp_4,flux_beyond_4,flux_mad_4,flux_skew_4,flux_mean_5,flux_std_5,flux_amp_5,flux_beyond_5,flux_mad_5,flux_skew_5
0,615,349.046051,-61.943836,320.796530,-51.753706,1,0.0000,0.0000,0.0000,NaN,...,-7.189853,0.431034,2.473048,0.0,-126.326202,2.333479,-6.340753,0.368421,2.311078,0.0
1,713,53.085938,-27.784405,223.525509,-54.460748,1,1.8181,1.6267,0.2552,45.4063,...,-5.291508,0.464286,1.291173,0.0,-4.904860,1.446335,-5.908843,0.428571,1.162656,0.0
2,730,33.574219,-6.579593,170.455585,-61.548219,1,0.2320,0.2262,0.0157,40.2561,...,1.411171,0.921569,0.077956,0.0,32.979248,0.404277,2.015506,0.941176,0.172627,0.0
3,745,0.189873,-45.586655,328.254458,-68.969298,1,0.3037,0.2813,1.1523,40.7951,...,1.635149,0.964286,0.019592,0.0,74.948265,0.347708,2.024899,0.945455,0.093099,0.0
4,1124,352.711273,-63.823658,316.922299,-51.059403,1,0.1934,0.2415,0.0176,40.4166,...,1.389328,0.982759,0.024281,0.0,86.915726,0.246614,1.380856,0.982456,0.058199,0.0


## ダミーデータでの解説
### 基本的な演算

In [3]:
# 以下のようなデータを用意する
dammy_dics = []
for i in range(5):
    for j in range(10):
        dammy_dics.append({"time": i, "category": j, "price": 10*i + j})

dammy_df = pd.DataFrame(dammy_dics)
dammy_df.head(10)

,time,category,price
0,0,0,0
1,0,1,1
2,0,2,2
3,0,3,3
4,0,4,4
5,0,5,5
6,0,6,6
7,0,7,7
8,0,8,8
9,0,9,9


In [4]:
# DataFrame.pivot_table()でクロス集計表を作れる
dammy_piv = dammy_df.pivot_table(index="time",
                                 columns="category",
                                 values="price",
                                 aggfunc="sum")
display(dammy_piv)

category,0,1,2,3,4,5,6,7,8,9
time,,,,,,,,,,
0,0,1,2,3,4,5,6,7,8,9
1,10,11,12,13,14,15,16,17,18,19
2,20,21,22,23,24,25,26,27,28,29
3,30,31,32,33,34,35,36,37,38,39
4,40,41,42,43,44,45,46,47,48,49


In [5]:
# pivot_tableは行列として計算することができる
# 各数値を二乗する
print("piv^2")
display(np.square(dammy_piv))

# スカラーで割る"
print("piv / 10")
display(dammy_piv / 10)

# pivot_table同士を足す
print("piv + piv^2")
display(dammy_piv + np.square(dammy_piv))

piv^2


category,0,1,2,3,4,5,6,7,8,9
time,,,,,,,,,,
0,0,1,4,9,16,25,36,49,64,81
1,100,121,144,169,196,225,256,289,324,361
2,400,441,484,529,576,625,676,729,784,841
3,900,961,1024,1089,1156,1225,1296,1369,1444,1521
4,1600,1681,1764,1849,1936,2025,2116,2209,2304,2401


piv / 10


category,0,1,2,3,4,5,6,7,8,9
time,,,,,,,,,,
0,0.0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9
1,1.0,1.1,1.2,1.3,1.4,1.5,1.6,1.7,1.8,1.9
2,2.0,2.1,2.2,2.3,2.4,2.5,2.6,2.7,2.8,2.9
3,3.0,3.1,3.2,3.3,3.4,3.5,3.6,3.7,3.8,3.9
4,4.0,4.1,4.2,4.3,4.4,4.5,4.6,4.7,4.8,4.9


piv + piv^2


category,0,1,2,3,4,5,6,7,8,9
time,,,,,,,,,,
0,0,2,6,12,20,30,42,56,72,90
1,110,132,156,182,210,240,272,306,342,380
2,420,462,506,552,600,650,702,756,812,870
3,930,992,1056,1122,1190,1260,1332,1406,1482,1560
4,1640,1722,1806,1892,1980,2070,2162,2256,2352,2450


In [6]:
# 列方向への集計
# axisを指定しないと自動的に列方向の集計になり、Seriesが返ってくる
display(dammy_piv.mean())

# pivot_tableに対してSeriesで計算するとと列方向にbroadcastされる
display(dammy_piv - dammy_piv.mean())

category
0    20.0
1    21.0
2    22.0
3    23.0
4    24.0
5    25.0
6    26.0
7    27.0
8    28.0
9    29.0
dtype: float64

category,0,1,2,3,4,5,6,7,8,9
time,,,,,,,,,,
0,-20.0,-20.0,-20.0,-20.0,-20.0,-20.0,-20.0,-20.0,-20.0,-20.0
1,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0
4,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0


In [7]:
# "行方向への集計も可能だが"
display(dammy_piv.mean(axis=1))

# いい感じにbroadcastしてくれない
print("piv - seires")
display(dammy_piv - dammy_piv.mean(axis=1))

# 転値を使うくらいしか良い方法が思い浮かばないので良い方法があれば教えてください
print("(piv.T - series).T")
display((dammy_piv.T - dammy_piv.mean(axis=1)).T)

time
0     4.5
1    14.5
2    24.5
3    34.5
4    44.5
dtype: float64

piv - seires


,0,1,2,3,4,5,6,7,8,9
time,,,,,,,,,,
0,-4.5,-13.5,-22.5,-31.5,-40.5,NaN,NaN,NaN,NaN,NaN
1,5.5,-3.5,-12.5,-21.5,-30.5,NaN,NaN,NaN,NaN,NaN
2,15.5,6.5,-2.5,-11.5,-20.5,NaN,NaN,NaN,NaN,NaN
3,25.5,16.5,7.5,-1.5,-10.5,NaN,NaN,NaN,NaN,NaN
4,35.5,26.5,17.5,8.5,-0.5,NaN,NaN,NaN,NaN,NaN


(piv.T - series).T


category,0,1,2,3,4,5,6,7,8,9
time,,,,,,,,,,
0,-4.5,-3.5,-2.5,-1.5,-0.5,0.5,1.5,2.5,3.5,4.5
1,-4.5,-3.5,-2.5,-1.5,-0.5,0.5,1.5,2.5,3.5,4.5
2,-4.5,-3.5,-2.5,-1.5,-0.5,0.5,1.5,2.5,3.5,4.5
3,-4.5,-3.5,-2.5,-1.5,-0.5,0.5,1.5,2.5,3.5,4.5
4,-4.5,-3.5,-2.5,-1.5,-0.5,0.5,1.5,2.5,3.5,4.5


### 差分

In [8]:
# piv.shift()でひとつ前の値をとれる
dammy_piv.shift(1)

category,0,1,2,3,4,5,6,7,8,9
time,,,,,,,,,,
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.0,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0
2,10.0,11.0,12.0,13.0,14.0,15.0,16.0,17.0,18.0,19.0
3,20.0,21.0,22.0,23.0,24.0,25.0,26.0,27.0,28.0,29.0
4,30.0,31.0,32.0,33.0,34.0,35.0,36.0,37.0,38.0,39.0


In [9]:
# これを活用すると、ひとつ前との差分をとることができる
dammy_piv - dammy_piv.shift(1)

category,0,1,2,3,4,5,6,7,8,9
time,,,,,,,,,,
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0
2,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0
3,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0
4,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0


### 移動平均

In [10]:
# rolling関数で、移動平均等をとることができる
# 以下のコードは自信を含めた三つの期間分の平均
dammy_piv.rolling(window=3, center=False).mean()

category,0,1,2,3,4,5,6,7,8,9
time,,,,,,,,,,
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,10.0,11.0,12.0,13.0,14.0,15.0,16.0,17.0,18.0,19.0
3,20.0,21.0,22.0,23.0,24.0,25.0,26.0,27.0,28.0,29.0
4,30.0,31.0,32.0,33.0,34.0,35.0,36.0,37.0,38.0,39.0


In [11]:
# shiftと組み合わせることで、一つ前からn個前までの平均といった特徴量を作ることができる
dammy_piv.rolling(window=3, center=False).mean().shift(1)

category,0,1,2,3,4,5,6,7,8,9
time,,,,,,,,,,
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,10.0,11.0,12.0,13.0,14.0,15.0,16.0,17.0,18.0,19.0
4,20.0,21.0,22.0,23.0,24.0,25.0,26.0,27.0,28.0,29.0


### ある時点までの合計を計算する

In [12]:
# cum〇〇系の関数はそれまでの合計を計算できる
# 合計
display(dammy_piv.cumsum())

category,0,1,2,3,4,5,6,7,8,9
time,,,,,,,,,,
0,0,1,2,3,4,5,6,7,8,9
1,10,12,14,16,18,20,22,24,26,28
2,30,33,36,39,42,45,48,51,54,57
3,60,64,68,72,76,80,84,88,92,96
4,100,105,110,115,120,125,130,135,140,145


In [13]:
# 上記までのテクニックを駆使すると、leak無しに時系列のmean_encodingができる
cum_sum = dammy_df.pivot_table(index="time",
                               columns="category",
                               values="price",
                               aggfunc="sum").cumsum()
cum_count = dammy_df.pivot_table(index="time",
                                 columns="category",
                                 values="price",
                                 aggfunc="count").cumsum()
cum_mean = cum_sum / cum_count
cum_mean_without_leakage = cum_mean.shift(1)
cum_mean_without_leakage

category,0,1,2,3,4,5,6,7,8,9
time,,,,,,,,,,
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.0,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0
2,5.0,6.0,7.0,8.0,9.0,10.0,11.0,12.0,13.0,14.0
3,10.0,11.0,12.0,13.0,14.0,15.0,16.0,17.0,18.0,19.0
4,15.0,16.0,17.0,18.0,19.0,20.0,21.0,22.0,23.0,24.0


# PLAsTiCCのデータを用いた実例

[Starter Kit](http://www.kaggle.com/michaelapers/the-plasticc-astronomy-starter-kit)  の3章のLightCurve内にある特徴量を計算する。

In [14]:
# データのロード
train_df = pd.read_csv("./input/training_set.scaled.csv", dtype={"object_id": np.uint32,
                                                           "mjd": np.float64,
                                                           "passband": np.uint8,
                                                           "flux": np.float32,
                                                           "flux_err": np.float32,
                                                           "detected": np.uint8})
train_meta_df = pd.read_csv('./input/training_set_metadata.scaled.csv')
test_meta_df = pd.read_csv('./input/test_set_metadata.scaled.csv')

In [15]:
# train_dfを集計してtrain_metaに結合したい
display(train_df.head())
display(train_meta_df.head())

,object_id,mjd,passband,flux,flux_err,detected
0,615,59750.4229,2,-544.810303,3.622952,1
1,615,59750.4306,1,-816.434326,5.553370,1
2,615,59750.4383,3,-471.385529,3.801213,1
3,615,59750.4450,4,-388.984985,11.395031,1
4,615,59752.4070,2,-681.858887,4.041204,1


,object_id,ra,decl,gal_l,gal_b,ddf,hostgal_specz,hostgal_photoz,hostgal_photoz_err,distmod,mwebv,target
0,615,349.046051,-61.943836,320.796530,-51.753706,1,0.0000,0.0000,0.0000,NaN,0.017,92
1,713,53.085938,-27.784405,223.525509,-54.460748,1,1.8181,1.6267,0.2552,45.4063,0.007,88
2,730,33.574219,-6.579593,170.455585,-61.548219,1,0.2320,0.2262,0.0157,40.2561,0.021,42
3,745,0.189873,-45.586655,328.254458,-68.969298,1,0.3037,0.2813,1.1523,40.7951,0.007,90
4,1124,352.711273,-63.823658,316.922299,-51.059403,1,0.1934,0.2415,0.0176,40.4166,0.024,90


In [16]:
print("train_meta: ", train_meta_df.shape)
print("test_meta: ", test_meta_df.shape)
print("テストデータは訓練データの{:.4}倍".format(test_meta_df.shape[0] / train_meta_df.shape[0]))

train_meta:  (7848, 12)
test_meta:  (3492890, 11)
テストデータは訓練データの445.1倍


## 愚直に計算する

In [17]:
# groupby無しに毎回取り出そうとするととてつもない時間がかかるので1/100だけ計算
bands = [train_df.passband == b for b in train_df.passband.unique()]
for id_ in tqdm(train_df.object_id.unique()[:78]):
    for band in bands:
        idx = train_df[(train_df.object_id == id_) & band].index
        flux, dflux = train_df.loc[idx, "flux"], train_df.loc[idx, "flux_err"]
        train_df.loc[idx, "flux_mean"] = np.sum(flux*np.square(flux/dflux))/np.sum(np.square(flux/dflux))
        fluxm = train_df.loc[idx, "flux_mean"]

        train_df.loc[idx, "flux_std"] = np.std(flux/fluxm, ddof = 1)
        train_df.loc[idx, "flux_amp"] = (np.max(flux) - np.min(flux))/fluxm
        train_df.loc[idx, "flux_mad"] = np.median(np.abs((flux - np.median(flux))/fluxm))
        train_df.loc[idx, "flux_beyond"] = sum(np.abs(flux - fluxm) > np.std(flux, ddof = 1))/len(flux)
# FIRST-AUTHOR: remove ML code
#         train_df.loc[idx, "flux_skew"] = skew(flux)
        train_df.loc[idx, "flux_skew"] = 0.0

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 78/78 [00:02<00:00, 30.50it/s]


これだとtrain_dataの処理でも一時間以上かかるので、その450倍もあるtest_dataを処理することはできない

## groupbyを使って計算する

In [18]:
# 2. groupbyを使って計算する
# FIRST-AUTHOR: remove extra display code
# tick = datetime.now()
train_df = pd.read_csv("./input/training_set.scaled.csv", dtype={"object_id": np.uint32,
                                                           "mjd": np.float64,
                                                           "passband": np.uint8,
                                                           "flux": np.float32,
                                                           "flux_err": np.float32,
                                                           "detected": np.uint8})
train_meta_df = pd.read_csv('./input/training_set_metadata.scaled.csv')
# FIRST-AUTHOR: remove extra display code
# tock = datetime.now()
# print("load_data: {} ms".format((tock - tick).seconds * 1000 + ((tock - tick).microseconds / 1000)))

# tick = datetime.now()

def agg_func(x):
    d = {}
    flux, dflux = x["flux"], x["flux_err"]
    flux_mean = np.sum(flux*np.square(flux/dflux))/np.sum(np.square(flux/dflux))
    d["flux_mean"] = flux_mean
    d["flux_std"] = np.std(flux/flux_mean, ddof = 1)
    d["flux_amp"] = (np.max(flux) - np.min(flux))/flux_mean
    d["flux_beyond"] = np.sum(np.abs(flux - flux_mean) > np.std(flux, ddof = 1))/flux.shape[0]
    d["flux_mad"] = np.median(np.abs((flux - np.median(flux))/flux_mean))
# FIRST-AUTHOR: remove ML code
#     d["flux_skew"] = skew(flux)
    d["flux_skew"] = 0.0
    return pd.Series(d, index = ["flux_mean", "flux_std", "flux_amp", "flux_mad", "flux_beyond", "flux_skew"])

result_df = train_df.groupby(["object_id", "passband"]).progress_apply(agg_func).reset_index()

colnames = ["flux_mean", "flux_std", "flux_amp", "flux_mad", "flux_beyond", "flux_skew"]
for j in range(6):
    train_meta_df = train_meta_df.merge(result_df.loc[result_df.passband == j, :]
                                                 .rename(columns={colname: "{}_{}".format(colname, j) for colname in colnames})
                                                 .drop("passband", axis=1),
                                        how="left",
                                        on=["object_id"])

# FIRST-AUTHOR: remove extra display code
# tock = datetime.now()
# tmp = print("total_processing: {} sec".format((tock - tick).seconds))
train_meta_df.head()

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 47088/47088 [01:23<00:00, 565.61it/s]


,object_id,ra,decl,gal_l,gal_b,ddf,hostgal_specz,hostgal_photoz,hostgal_photoz_err,distmod,...,flux_amp_4,flux_mad_4,flux_beyond_4,flux_skew_4,flux_mean_5,flux_std_5,flux_amp_5,flux_mad_5,flux_beyond_5,flux_skew_5
0,615,349.046051,-61.943836,320.796530,-51.753706,1,0.0000,0.0000,0.0000,NaN,...,-7.189853,2.473048,0.431034,0.0,-126.326202,2.333479,-6.340753,2.311078,0.368421,0.0
1,713,53.085938,-27.784405,223.525509,-54.460748,1,1.8181,1.6267,0.2552,45.4063,...,-5.291508,1.291173,0.464286,0.0,-4.904860,1.446335,-5.908843,1.162656,0.428571,0.0
2,730,33.574219,-6.579593,170.455585,-61.548219,1,0.2320,0.2262,0.0157,40.2561,...,1.411171,0.077956,0.921569,0.0,32.979244,0.404277,2.015506,0.172627,0.941176,0.0
3,745,0.189873,-45.586655,328.254458,-68.969298,1,0.3037,0.2813,1.1523,40.7951,...,1.635149,0.019592,0.964286,0.0,74.948257,0.347708,2.024899,0.093099,0.945455,0.0
4,1124,352.711273,-63.823658,316.922299,-51.059403,1,0.1934,0.2415,0.0176,40.4166,...,1.389328,0.024281,0.982759,0.0,86.915718,0.246614,1.380856,0.058199,0.982456,0.0


一時間以上かかった処理を二分半で終えることができたが、testデータだと900分 = 15時間かかるのでまだまだ高速化したい

## pivot_tableを使って計算する

In [19]:
# 欠損値が含まれることに注意する

# FIRST-AUTHOR: remove extra display code
# tick = datetime.now()
train_df = pd.read_csv("./input/training_set.scaled.csv", dtype={"object_id": np.uint32,
                                                           "mjd": np.float64,
                                                           "passband": np.uint8,
                                                           "flux": np.float32,
                                                           "flux_err": np.float32,
                                                           "detected": np.uint8})
train_meta_df = pd.read_csv('./input/training_set_metadata.scaled.csv')
# FIRST-AUTHOR: remove extra display code
# tock = datetime.now()
# print("load_data: {} ms".format((tock - tick).seconds * 1000 + ((tock - tick).microseconds / 1000)))

# tick = datetime.now()

# pivot_tableのindexをrankを用いて作成する
train_df["rank"] = train_df.groupby(["object_id", "passband"])["mjd"].rank()

flux = train_df.pivot_table(columns=["object_id", "passband"],
                            index="rank",
                            values="flux",
                            aggfunc="mean")
dflux = train_df.pivot_table(columns=["object_id", "passband"],
                             index="rank",
                             values="flux_err",
                             aggfunc="mean")

# 列にNaNが含まれるので扱いに注意する
flux_mean = np.sum(flux*np.square(flux/dflux), axis=0)/np.sum(np.square(flux/dflux), axis=0)
flux_std = np.std(flux/flux_mean, ddof = 1, axis=0)
flux_amp = (np.max(flux, axis=0) - np.min(flux, axis=0))/flux_mean
flux_mad = np.nanmedian(np.abs((flux - np.nanmedian(flux, axis=0))/flux_mean), axis=0) # array
flux_beyond = np.sum(np.abs(flux - flux_mean) > np.std(flux, ddof = 1, axis=0), axis=0)/flux.count()
# FIRST-AUTHOR: remove ML code
# flux_skew = skew(flux, nan_policy="omit", axis=0)  # masked_array
flux_skew = 0.0

result_df = pd.concat([flux_mean.reset_index(name="flux_mean"),
                      flux_std.reset_index(name="flux_std").iloc[:, 2:],
                      flux_amp.reset_index(name="flux_amp").iloc[:, 2:],
                      flux_beyond.reset_index(name="flux_beyond").iloc[:, 2:]], axis=1)
result_df["flux_mad"] = flux_mad
result_df["flux_skew"] = flux_skew
colnames = ["flux_mean", "flux_std", "flux_amp", "flux_beyond", "flux_mad", "flux_skew"]

for j in range(6):
    train_meta_df = train_meta_df.merge(result_df.loc[result_df.passband == j, :]
                                                 .rename(columns={colname: "{}_{}".format(colname, j) for colname in colnames})
                                                 .drop("passband", axis=1),
                                        how="left",
                                        on=["object_id"])
# FIRST-AUTHOR: remove extra display code
# tock = datetime.now()
# print("processing_time: {} sec".format((tock - tick).seconds))

train_meta_df.head()

,object_id,ra,decl,gal_l,gal_b,ddf,hostgal_specz,hostgal_photoz,hostgal_photoz_err,distmod,...,flux_amp_4,flux_beyond_4,flux_mad_4,flux_skew_4,flux_mean_5,flux_std_5,flux_amp_5,flux_beyond_5,flux_mad_5,flux_skew_5
0,615,349.046051,-61.943836,320.796530,-51.753706,1,0.0000,0.0000,0.0000,NaN,...,-7.189853,0.431034,2.473048,0.0,-126.326202,2.333479,-6.340753,0.368421,2.311078,0.0
1,713,53.085938,-27.784405,223.525509,-54.460748,1,1.8181,1.6267,0.2552,45.4063,...,-5.291508,0.464286,1.291173,0.0,-4.904860,1.446335,-5.908843,0.428571,1.162656,0.0
2,730,33.574219,-6.579593,170.455585,-61.548219,1,0.2320,0.2262,0.0157,40.2561,...,1.411171,0.921569,0.077956,0.0,32.979248,0.404277,2.015506,0.941176,0.172627,0.0
3,745,0.189873,-45.586655,328.254458,-68.969298,1,0.3037,0.2813,1.1523,40.7951,...,1.635149,0.964286,0.019592,0.0,74.948265,0.347708,2.024899,0.945455,0.093099,0.0
4,1124,352.711273,-63.823658,316.922299,-51.059403,1,0.1934,0.2415,0.0176,40.4166,...,1.389328,0.982759,0.024281,0.0,86.915726,0.246614,1.380856,0.982456,0.058199,0.0


groupbyで二分半ほどかかっていた処理を、わずか4秒で処理することができた!!

testデータは大きすぎるので一度に計算しようとするとメモリに乗り切らないが、私の環境(RAM 32GB)だと10分割して計算しおおよそ30分くらいで処理が終わった。
